In [1]:
# -*- coding: utf-8 -*-
import json
import pandas as pd
from tracking_grants.utils.logging import logger
from tracking_grants import references_f, articles_f, matched_articles_f

In [2]:
# Load files
refs = pd.read_csv(references_f)
matches = pd.DataFrame.from_dict(json.loads(matched_articles_f.read_text()))

In [3]:
# Merge references and matched responses from crossref by reference ID
merged = refs.merge(matches[['score', 'DOI']], left_on="reference_id", right_index=True)

# Drop all references without a match
merged = merged[merged.DOI.notna()]

In [4]:
merged = merged[['reference_id', 'grant_id', 'program', 'score', 'DOI']].drop_duplicates()

In [6]:
merged.to_csv(articles_f, index=False)

## Matching

In [32]:
from tracking_grants import (matched_articles_f, one_ref_per_line,
                             reference_matcher, references_f, data_dir)

from pathlib import Path

In [33]:
n_refs = 0
with open(one_ref_per_line) as rf:
    n_refs = sum(1 for line in rf)

In [55]:
batch_size = 500
batches = list(range(0, n_refs, batch_size))
tempfiles = [data_dir / f"interim/{i}.txt" for i in range(0,len(batches))]
outfiles = [data_dir / f"interim/{i}.json" for i in range(0,len(batches))]

In [45]:
for f in tempfiles:
    f.open("w")
    
with open(one_ref_per_line) as rf:
    counter = 0
    file_count = 0
    for line in rf:
        if counter == batch_size:
            file_count = file_count + 1
            counter = 0
            
        with tempfiles[file_count].open('a+') as file:
            if counter==499:
                file.write(line.strip())
            else:
                file.write(line)
            
        counter = counter + 1

In [56]:
import json

In [66]:
# merge all outfiles
output = []
for fout in outfiles:
    d = json.loads(fout.read_text())
    output = output + d

with matched_articles_f.open("w") as f:
    json.dump(output, f)

In [67]:
for fin, fout in zip(tempfiles, outfiles):
    fin.unlink()
    fout.unlink()

In [52]:
for f in tempfiles:
    print()

0.json
1.json
2.json
3.json
4.json
5.json
6.json
7.json
8.json
9.json
10.json
11.json
12.json
13.json
14.json
15.json
16.json
17.json
18.json
19.json
